In [35]:
import pandas as pd
import numpy as np
# from tqdm.notebook import tqdm
# from tqdm.autonotebook import tqdm
# from tqdm import tqdm
# blah blah your code errored
# tqdm._instances.clear()


In [36]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60

NUM_FLOW = 20
DATA_NAME = "harmonic3"
TOPO_NAME = "0"

task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:NUM_FLOW]
network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)
for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

## 1. Model

Network model:

$\left\langle\left[v_{a}, v_{b}\right] . s,\left[v_{a}, v_{b}\right] . d,\left[v_{a}, v_{b}\right] . m t,\left[v_{a}, v_{b}\right] . c\right\rangle$

Network model

In [37]:
net_var = {}

for _, row in network.iterrows():
    net_var.setdefault(row['link'], {})
    net_var[row['link']]['tproc'] = row['t_proc']
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

In [38]:
## Create mapping from Link to index
link_to_index = {}
index_to_link = {}

counter = 0
for _, row in network.iterrows():
    link = row['link']
    link_to_index[link] = counter
    index_to_link[counter] = link
    counter += 1

Task model

In [39]:
task_attr = {}
task_var = {}
for k, row in task.iterrows():
    task_attr.setdefault(k, {})
    task_var.setdefault(k, {})
    
    
    ## f_k
    task_attr[k]['sen'] = int(row['src'])
    task_attr[k]['rec'] = int(eval(row['dst'])[0])
    task_attr[k]['n'] = int(row['size'])
    task_attr[k]['p'] = int(row['period'] )
    task_attr[k]['d'] = int(row['deadline'])
    task_attr[k]['TD'] = task_attr[k]['n'] * 8
    
    task_var[k]['priority'] = 0
    task_var[k]['e2eD'] = 0
    task_var[k]['arr'] = 0
    task_var[k]['IT'] = 0
    task_var[k]['r'] = None
    

In [40]:
def find_all_paths(graph, start, end, path=[]):
        path = path + [start]
        if start == end:
            return [path]
        paths = []
        for node in set(np.reshape(np.argwhere(graph[start] > 0),  -1)):
            if node not in path:
                newpaths = find_all_paths(graph, node, end, path)
                for newpath in newpaths:
                    paths.append(newpath)
        return paths     

## 1. Assign prority

$CC_f=f_{PT} + f_{lj}.TD$

In [41]:
for i in task_var:
    paths = find_all_paths(net, task_attr[i]['sen'], task_attr[i]['rec'])
    task_attr[i]['paths'] = paths
    task_var[i]['priority'] = max([len(x) for x in paths])

In [42]:
task_order = sorted(list(task_attr.keys()), key= lambda x: task_var[x]['priority'], reverse=True)

## 2. Scheduler

In [43]:
def match_time(t, sche) -> int:
    '''
    Use binary search to quickly find the posistion of GCL
    '''
    if not sche:
        return -1
    gate_time = [x[0] for x in sche]
    left = 0
    right = len(sche) - 1
    if gate_time[right] <= t < sche[-1][1]:
        return right
    elif sche[-1][1] <= t:
        return -2
    elif t < gate_time[0]:
        return -1
    
    while True:
        median = (left + right) // 2
        if right - left <= 1:
            return left
        elif gate_time[left] <= t < gate_time[median]:
            right = median
        else:
            left = median

In [44]:
def FindET(task, GCL, path):
    arr = (len(path) - 1) * (max(network['t_proc']) + task_attr[task]['TD'])
    for it in range(0, task_attr[task]['p'] - arr + 1):
        _last_hop_end = it
        for i, v in enumerate(path[:-1]):
            link_flag = True
            link = str((v, path[i+1]))
            _current_hop_start = _last_hop_end
            _last_hop_end = _current_hop_start + max(network['t_proc']) + task_attr[task]['TD']
            for alpha in range(0,  int(LCM / task_attr[task]['p'])):
                ## REQUIRE TO BE MERGED INTO ORIONAL GITREPO
                match_start = match_time(_current_hop_start + alpha * task_attr[task]['p'], GCL[link])
                match_end =  match_time(_last_hop_end - max(network['t_proc']) + alpha * task_attr[task]['p'], GCL[link])
                if match_start == -1 and GCL[link] and _last_hop_end - max(network['t_proc']) + alpha * task_attr[task]['p'] > GCL[link][0][0]:
                    link_flag = False
                    break
                if match_start == -2 and GCL[link] and _current_hop_start + alpha * task_attr[task]['p'] < GCL[link][-1][1]:
                    link_flag = False
                    break
                if match_start >= 0 and (match_start != match_end or (GCL[link] and
                GCL[link][match_start][1] > _current_hop_start + alpha * task_attr[task]['p'])):
                    link_flag = False
                    break
            if link_flag:
                ## This link is available and continue to next link
                continue
            else:
                ## Break the search on this IT time
                break
        else:
            return it
    return -1

In [45]:
def Scheduler(task):
    global GCL, OFFSET
    task_var[task]['arr'] = 0
    for r in task_attr[task]['paths']:
        IT = FindET(task, GCL, r)
        arr = (len(r) - 1) * (max(network['t_proc']) + task_attr[task]['TD'])
        if IT == -1 or arr > task_attr[task]['d']:
            continue
        if task_var[task]['arr'] == 0 or arr < task_var[task]['arr']:
            task_var[task]['arr'] = arr
            task_var[task]['IT'] = IT
            task_var[task]['r'] = r
    if task_var[task]['arr'] == 0:
        return False
        
    _last_hop_end = task_var[task]['IT']
    for i, v in enumerate(task_var[task]['r'][:-1]):
        link = str((v, task_var[task]['r'][i+1]))
        _current_hop_start = _last_hop_end
        _last_hop_end = _current_hop_start + max(network['t_proc']) + task_attr[task]['TD']
        for alpha in range(0, int(LCM / task_attr[task]['p'])):
            GCL[link].append([_current_hop_start + alpha * task_attr[task]['p'], 
                              _last_hop_end - max(network['t_proc']) + alpha * task_attr[task]['p'],
                              0])
            # if link == "(8, 0)":
            #     print(GCL[link])
        if i == 0:
            OFFSET[task] = _current_hop_start
    return True

In [46]:
GCL = {}
OFFSET = {}
for link in link_to_index:
    GCL.setdefault(link, [])

In [47]:
for i in task_order:
    for link in GCL:
        GCL[link] = sorted(GCL[link], key=lambda x:x[0], reverse=False)
    success = Scheduler(i)
    if not success:
        print("false")
        break

In [48]:
# GCL["(8, 0)"] = GCL["(8, 0)"][:1]

# if match_start > 0 and (match_start != match_end or (GCL[link] and
#                 GCL[link][match_start][1] > _current_hop_start + alpha * task_attr[task]['p'])):

# match_time(0, GCL["(8, 0)"])

# match_time(80, GCL["(8, 0)"])

## Output schedule

In [49]:
GCL_out = []
for link in GCL:
    [GCL_out.append([eval(link),
               row[2],
               row[0] * macrotick, 
               row[1] * macrotick, 
               LCM * macrotick]) for row in GCL[link]]
GCL = GCL_out
# with open('SIGBED19-GCL.txt', 'w') as f:
#     f.write(str(GCL))

In [50]:
## Offset
OFFSET_out = []
for i in OFFSET:
    OFFSET_out.append(
        [i, 0, (task.loc[i,'period'] - OFFSET[i]) * macrotick]
    )    
OFFSET = OFFSET_out

In [51]:
ROUTE = []
for i, row in task.iterrows():
    route = task_var[i]['r']
    for h, v in enumerate(route[:-1]):
        ROUTE.append(
            [i, (v, route[h + 1])]
        )

In [52]:
QUEUE = []
for i in task_var:
    route = task_var[i]['r']
    for h, v in enumerate(route[:-1]):
        QUEUE.append([i, 0, (v, route[h + 1]), 0])

In [53]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("SIGBED2019-%s-%d-%s-GCL.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("SIGBED2019-%s-%d-%s-OFFSET.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("SIGBED2019-%s-%d-%s-ROUTE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("SIGBED2019-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)